In [1]:
import pandas as pd
import math

from io import StringIO
from IPython.lib.pretty import pprint

In [2]:
datastr = StringIO("""
Pokemon	Label	Level	SpdMod	IngMod	BerMod	SklMod	Ingredients	Berries
Pikachu	08/17 Pikachu Lvl: 13, Spd: +	13	+				2	15
Pikachu	08/25 Pikachu Lvl: 5, Ing: +	5		+			2	13
Pikachu	08/14 Pikachu Lvl: 12, Spd: M, Ing: +	12	M	+			3	16
Pikachu	08/09 Pikachu Lvl: 5, Ing: -, Skl: +	5		-		+	1	15
""")

data = pd.read_csv(datastr, sep='\t', header=0)
data = data.set_index (["Pokemon","Label"])
data = data.fillna("") 
data.sort_index(axis=0, ascending=True, inplace=True)
data.head(30)


data.to_pickle('./data/observations.pickle')

In [3]:
def computeBonusAndSpeed(individual, species, output):

    berryBonus = 2.0 if species["Specialty"] == 'Berries' else 1.0
    ingBonus = 2.0 if species["Specialty"] == 'Ingredients' else 1.0
    
    if individual["BerMod"].find("S") > -1: berryBonus += 1.0
    
    output["IngBonus"] = ingBonus
    output["BerryBonus"] = berryBonus
        
    period = species["Period"]
    if individual["SpdMod"].find("S") > -1: period *= 0.93
    if individual["SpdMod"].find("M") > -1: period *= 0.86
    if individual["SpdMod"].find("+") > -1: period *= 0.9
    if individual["SpdMod"].find("-") > -1: period *= 1.1
    period /= (1 + (individual["Level"]-1)*2.0/1000)
    #period = math.floor(period)
    
    output["Period"] = period
    output["Help3h"] = (3*60*60)/period
    
    delta = 1.0
    if individual["IngMod"].find("S") > -1: delta *= 1.18
    if individual["IngMod"].find("M") > -1: delta *= 1.36
    if individual["IngMod"].find("+") > -1: delta *= 1.25
    if individual["IngMod"].find("-") > -1: delta /= 1.25
    
    output["IngDelta"] = delta
    

In [4]:
pokedex = pd.read_pickle('./data/pokedex.pickle')
pokedex_dict  = pokedex.to_dict(orient='index', index=True)
data_list  = data.reset_index().to_dict(orient='records', index=True)

In [5]:
for individual in data_list:   
    
    species = pokedex_dict[individual['Pokemon']]
    computeBonusAndSpeed(individual, species, individual)

In [6]:
computed = pd.DataFrame.from_records(data_list, index=['Pokemon',"Label"])
computed.sort_index(axis=0, ascending=True, inplace=True)

computed.to_pickle('./data/computed.pickle')
computed.head(30)

Level SpdMod IngMod BerMod  \
Pokemon Label                                                               
Pikachu 08/09 Pikachu Lvl: 5, Ing: -, Skl: +       5             -          
        08/14 Pikachu Lvl: 12, Spd: M, Ing: +     12      M      +          
        08/17 Pikachu Lvl: 13, Spd: +             13      +                 
        08/25 Pikachu Lvl: 5, Ing: +               5             +          

                                              SklMod  Ingredients  Berries  \
Pokemon Label                                                                
Pikachu 08/09 Pikachu Lvl: 5, Ing: -, Skl: +       +            2       15   
        08/14 Pikachu Lvl: 12, Spd: M, Ing: +                   3       16   
        08/17 Pikachu Lvl: 13, Spd: +                           2       15   
        08/25 Pikachu Lvl: 5, Ing: +                            2       13   

                                               IngBonus  BerryBonus  \
Pokemon Label                                                         
Pikachu 08/09 Pikachu Lvl: 5, Ing: -, Skl: +        1.0         2.0   
        08/14 Pikachu Lvl: 12, Spd: M, Ing: +       1.0         2.0   
        08/17 Pikachu Lvl: 13, Spd: +               1.0         2.0   
        08/25 Pikachu Lvl: 5, Ing: +                1.0         2.0   

                                                    Period    Help3h  IngDelta  
Pokemon Label                                                                   
Pikachu 08/09 Pikachu Lvl: 5, Ing: -, Skl: +   2678.571429  4.032000      0.80  
        08/14 Pikachu Lvl: 12, Spd: M, Ing: +  2272.015656  4.753488      1.25  
        08/17 Pikachu Lvl: 13, Spd: +          2373.046875  4.551111      1.00  
        08/25 Pikachu Lvl: 5, Ing: +           2678.571429  4.032000      1.25

In [7]:
"""
Pokemon	Label	Level	SpdMod	IngMod	BerMod	SklMod	Ingredients	Berries
Raichu	08/11 Raichu Lvl: 12, Spd: +	12	+				3	19 
Raichu	08/14 Raichu Lvl: 16, Spd: +, Ber: S, Skl: -	16	+		S	-	3	29
Pichu	08/08 Pichu Lvl: 3	3					1	8
"""

'\nPokemon\tLabel\tLevel\tSpdMod\tIngMod\tBerMod\tSklMod\tIngredients\tBerries\nRaichu\t08/11 Raichu Lvl: 12, Spd: +\t12\t+\t\t\t\t3\t19 \nRaichu\t08/14 Raichu Lvl: 16, Spd: +, Ber: S, Skl: -\t16\t+\t\tS\t-\t3\t29\nPichu\t08/08 Pichu Lvl: 3\t3\t\t\t\t\t1\t8\n'